In [1]:
from load_utils import prepare_data
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from tqdm.notebook import tnrange, tqdm_notebook

In [2]:
q, a, pairs, vector = prepare_data('test', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading test -------
Read 4041 sentence pairs
Counting words
Counted words:
In questions: 6723 words
In answers: 4670 words


In [3]:
def epoch_train(model, optimizer, batch_size, pairs):
    
    # Set the model in train mode
    model.train()
    
    # Gets number total number of rows for training
    n_records = len(pairs)
    
    # Shuffle the row indexes 
    perm = np.random.permutation(n_records)
    
    st = 0
    
    while st < n_records:
        
        ed = st + batch_size if (st + batch_size) < n_records else n_records
    
        encoder_in, decoder_in = to_batch_sequence(pairs, st, ed, perm, device)

        # Calculate outputs and loss
        output_values, loss = model(encoder_in, decoder_in)
        
        # Clear gradients (pytorch accumulates gradients by default)
        optimizer.zero_grad() 

        # Backpropagation & weight adjustment
        loss.backward()
        optimizer.step()
        
        st = ed

print(f"Optimization ended successfully")

Optimization ended successfully


In [4]:
perm = np.random.permutation(len(pairs))
perm 

array([2115,  245, 1710, ..., 1655, 1752,  370])

In [5]:
def to_batch_sequence(pairs, st, ed, perm, device):
    
    encoder_in = []
    decoder_in = []
    for i in range(st, ed):
        
        pair_batch = pairs[perm[i]]
        encoder_in.append(pair_batch[0])
        decoder_in.append(pair_batch[1])
    
    encoder_in = [[q.word2index.get(idx) for idx in encoder_in[row].split()] for row in range(len(encoder_in))]
    decoder_in = [[q.word2index.get(idx) for idx in decoder_in[row].split()] for row in range(len(decoder_in))]
    
    encoder_lengths = [len(row) for row in encoder_in]
    decoder_lengths = [len(row) for row in decoder_in]
    
    max_encoder_length = max(encoder_lengths)
    max_decoder_length = max(decoder_lengths)
    
    encoder_in_tensor = torch.zeros(ed, max_encoder_length, device=device, dtype=torch.float)
    decoder_in_tensor = torch.zeros(ed, max_decoder_length, device=device, dtype=torch.float)
    
    for i, seq in enumerate(encoder_in):
        for t, word in enumerate(seq):
            if type(word) == int:
                encoder_in_tensor[i, t] = word 
                
    for i, seq in enumerate(decoder_in):
        for t, word in enumerate(seq):
            if type(word) == int:
                decoder_in_tensor[i, t] = word 
        
    return encoder_in_tensor, decoder_in_tensor

In [6]:
pair_batch = pairs[perm[0]]

In [7]:
pair_batch[0]

'yeah.if we share the cab it ll be cheaper.besides that your chinese is better than mine.the drivers usually have difficulty understanding me even though i practice my pronunciation carefully .'

In [8]:
pair_batch[1]

'yes that s right . after graduating from high school in new york i attended york university in the uk . my major was english and my minor was business studies . i completed my ba in .'

In [9]:
encoder_in, decoder_in = to_batch_sequence(pairs, 0, 32, perm, device='cpu')

In [10]:
encoder_in

tensor([[5947.,   35.,  498.,  ...,    0.,    0.,    0.],
        [3143., 5134., 1331.,  ...,    0.,    0.,    0.],
        [5074.,  768.,   14.,  ...,    0.,    0.,    0.],
        ...,
        [  74.,   12.,  753.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [6062.,   36.,   66.,  ...,    0.,    0.,    0.]])

In [11]:
[len(row) for row in encoder_in]

[36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36]

In [12]:
zero = torch.zeros(32, 33, device='cpu', dtype=torch.float)
zero

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [13]:
for i, seq in enumerate(encoder_in):
    for t, word in enumerate(seq):
        if type(word) == int:
            zero[i, t] = word 

zero

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
for i, seq in enumerate(encoder_in):
    for t, word in enumerate(seq):
        print(type(word))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor

In [20]:
n = torch.zeros(64-32, 20, device='cuda', dtype=torch.float)

In [22]:
n.size()[0]

32